### 1. What is the precision of our sensors? Assume at 5 of the 10 locations, we co-located two of our sensors. 

Precision depicts the statistical variability of measurements and will be limited to the 5 sites that host 2 Clarity sensors. In order to calculate a generalized precision score, the difference between each concurring Clarity sensor at the 5 reference sites will need to be computed for each 15 minute interval.

t = number of days in operation

number of observations in precision dataset = $5*4*24*t = 480t$

The overall precision of Clarity’s sensors will be the standard deviation of the $480t$ difference measurements divided by the square root of the number of observations.

$precision = \dfrac{\sqrt{\dfrac{1}{n-1} \sum_{k=1}^n (Clarity_{k1} - Clarity_{k2})^2}}{\sqrt{n}}$

#### Fun with simulations

In [33]:
'''Generate some random number pairs from two normal distributions with
slightly different standard deviations'''

# Import python dependencies
import numpy as np
import random
import pandas as pd

#Generate random number pairs from slightly different normal distributions. 
n = 480*60
C1 = [random.gauss(10, .6) for _ in range(n)]
C2 = [random.gauss(10, .65) for _ in range(n)]

#Create and display a sample table
precision_dataframe = pd.DataFrame({'Sensor_1': C1, 'Sensor_2': C2})
print(precision_dataframe.head(10), '\n\n')

#Calculate precision
precision = sum([np.sqrt((1/(n-1))*(C1[i]-C2[i])**2) for i in range(n)])/np.sqrt(n)
print("Precision: ",precision)

    Sensor_1   Sensor_2
0  10.759044   9.864354
1  10.834644  10.271047
2  10.850494  10.138717
3   8.949988   9.885044
4   8.811633  10.502993
5  10.286469   9.636076
6   9.702521  10.520662
7   9.854163  10.231671
8  10.102289   9.927270
9   9.335530   9.909060 


Precision:  0.707630272066


### 2. What is the accuracy of our sensors?

Sensors are accurate if the difference between a measurement and a “true value” is low. Since we are comparing numerical estimates, accuracy is best expressed in terms of **absolute or percent error.**

Given that there is a total of 15 experimental sensors that can be compared to ground truth measurements every hour, there will be 360t observations that we can use to evaluate accuracy. 

t = number of days in operation

number of observations in accuracy dataset = $15*24*t = 360t$


According to the central limit theorem, differences in sensor readings will be normally distributed and can be summarized as a mean with a 95% confidence interval. This is a measure of the mean absolute error, or the degree to which Clarity sensors are inaccurate in terms of measurement units.

**absolute error** $= \dfrac{1}{n-1} \sum_{k=1}^n |Reference_{k} - Clarity_{k}| \pm 1.96(STDEV(|Reference_{k} - Clarity_{k}|)$


Absolute error can be normalized in terms of percent error.

**percent error** $= \dfrac{\dfrac{1}{n-1} \sum_{k=1}^n |Reference_{k} - Clarity_{k}|}{
\dfrac{1}{n-1} \sum_{k=1}^n Reference_{k}} \pm 1.96(STDEV(\dfrac{|Reference - Clarity|}{Reference})$


In [52]:
#Generate random number pairs for Clarity and reference sensors. 
n = 360*60
clarity_sensors = [random.gauss(10, .6) for _ in range(n)]
reference_sensors = [random.gauss(13, .32) for _ in range(n)]

#Create an example table and calculate absolute error and percent error
accuracy_dataframe = pd.DataFrame({'Clarity': clarity_sensors, 'Reference': reference_sensors})
accuracy_dataframe['Absolute_Error'] = abs(accuracy_dataframe['Clarity']-accuracy_dataframe['Reference'])
accuracy_dataframe['Pct_Error'] = abs(accuracy_dataframe['Clarity']-accuracy_dataframe['Reference'])/accuracy_dataframe['Reference']

print(accuracy_dataframe.head(10), '\n\n')

#Compute a 95% Confidence Interval for mean absolute error and mean percent error
mean_absolute_error = accuracy_dataframe['Absolute_Error'].mean()
mean_absolute_error_MOE =1.96*accuracy_dataframe['Absolute_Error'].std()
print('mean absolute error = ', mean_absolute_error, '+/-', mean_absolute_error_MOE)

mean_pct_error = accuracy_dataframe['Pct_Error'].mean()
mean_pct_error_MOE = 1.96*accuracy_dataframe['Pct_Error'].std()
print('mean percent error = ', mean_pct_error, '+/-', mean_pct_error_MOE)

     Clarity  Reference  Absolute_Error  Pct_Error
0   9.830261  12.417301        2.587039   0.208342
1  10.342444  12.397132        2.054688   0.165739
2  10.261964  12.971996        2.710032   0.208914
3   9.940633  12.675248        2.734615   0.215744
4  10.619086  13.129210        2.510124   0.191186
5   9.928284  13.366421        3.438137   0.257222
6  10.184502  12.677488        2.492986   0.196647
7  10.568586  13.144947        2.576361   0.195996
8   9.954521  13.252518        3.297997   0.248858
9   9.758751  13.042618        3.283867   0.251780 


mean absolute error =  3.0012495903719394 +/- 1.3333102904344172
mean percent error =  0.2303927162274987 +/- 0.09797096405369289


#### Are the measurment differences between Clarity and reference sensors statistically significant?

null-hypothesis: There is no difference between Clarity sensors and the reference sensors.

alternative-hypothesis: There is a difference between Clarity sensors and the reference sensors.

In [55]:
from scipy import stats
stats.ttest_ind(accuracy_dataframe['Reference'], accuracy_dataframe['Clarity'])

Ttest_indResult(statistic=647.83542781550659, pvalue=0.0)

With a p-value of zero, there is evidence to reject the null-hypothesis.
The randomly generated numbers for the Clarity sample are different from the numbers generated for the reference sample.

### 3. Give some approaches you would use to create a predictive model based on these 10 locations that can improve our sensor's readings compared to the reference monitor. This model will be  used to calibrate sensors in the network that are not next to a reference monitor (future deployment). 

Calibration is a regression problem. The objective is to train a model that will map estimates from Clarity sensors to control sensor readings while minimizing error. Assuming that the Clarity and control sensors have a linear relationship, some configuration of a multivariate linear regression model will be appropriate and will allow us to infer the importance of the 3 variables that Clarity sensors track.

If the relationship between the Clarity and reference sensor measurements is non-linear, a regression model can be improved by tweaking hyper-parameters and engineering new features (e.g. log transformations, basis expansion, implementing regression splines, etc.). If the data is highly non-linear and calls for a large feature set, regularization techniques such as ridge, lasso, elastic net can be used to penalize model complexity and prevent overfitting. Decision trees and the Perceptron algorithm could be considered as candidate models if the data is extremely non-linear, though I would be hesitant to use these models since they are prone to overfitting and are more difficult to interpret than linear regression. 
![linear regrssion](linear_regression.png)

**This is beyond the scope of initial sensor calibration, but can help sensors localize and calibrate continuously:**

Since this is time-series data, I can include temporal lags as features in an ARMA model (auto-regressive moving average). In other words, each row in the dataset will have features for the PM reading of that sensor 15 minutes in the past, 30 past,45 minutes past, etc. and can extrapolate future predictions. I would have the model predict PM readings at future time steps and have the model evaluate prediction errors as live data becomes available. 

#### Cross-Validation

Candidate models will be evaluated in terms of Mean Square Error (or an adjusted RSS term such as BIC), and Adjusted R^2 over k-steps of cross-validation on the training set. I would set aside 20% aside as testing data that will be reserved for evaluating the final model. I would set aside another 20% of the data for model checking as I test out a range of candidate models. The remaining 60% of the dataset will be used to train and develop the candidate models.

![holdout](holdout.png)


Since there are 15 Clarity sensors and $t = daysActive$, the calibration dataset will consist of 360t observations that map the concurrent output of a Clarity sensor to control sensor.

I would set aside 20%, or 72n observations aside as testing data that will be reserved for evaluating the final model. That leaves 288n rows of data for model development. 

I would set aside another 20% of the data for model checking as I test out a range of candidate models.

The remaining 60%, or 216n rows of the dataset will be used to train and develop the candidate models.

![cross_validation](cross_val.png)


### 4. Given a network of 500 sensors, what kind of visualizations and plots would you try?

Starting with the basics, I think the first task would be to interpolate particulate matter estimates across geographic space moment by moment using your sensor data. Probabilistic processes such as **kriging** are standard practice in geostatistics.

![kriging](kriging.png)

Chloroplath maps can be used to aggregate readings within local wards.

![chloropleth](chloropleth.png)

I could also plot the density of anomalous readings that occur over a period of time. If a sensor's time-series model gets a PM reading that is outside of the margin of error of its prediction, the map might look something like this:

In [75]:
import IPython
iframe = '<iframe width="100%" height="520" frameborder="0"\
src="https://amarinusha.carto.com/viz/3d91e128-ae90-11e6-b309-\
0ef24382571b/embed_map"\
allowfullscreen webkitallowfullscreen mozallowfullscreen \
oallowfullscreen msallowfullscreen></iframe>'
IPython.display.HTML(iframe)


I think the most compelling visual would be a timelapse that shows the generation, movement, and dissipation of pollution fog over different time steps (I would be especially interested in seeing patterns within a 24 hour period). 

From a modeling perspective, I would be really interested in forecasting air pollution with models that account for moisture, temperature, land cover, and biomass. Markov Random Fields are a type of probabilistic graphical model would be well suited for this task.

![markov random field](mrf.png)

Each cell in a grid contains a feature space of variables like the ones I mentioned above. The network also has hidden layers that contain past information. The value predicted for each cell is dependent on the the values of its neighbors and its past.

With a larger and more distributed network of sensors, we might be able to generate enough data to train a Convolutional Neural Net (CNN) to predict PM estimates using satellite imagery. Satellites capture spectral data from light that is reflected by atmospheric particles and objects. Deep learning is great for learning complex and hierarchical patterns in image data.

Jean et al. out of Stanford recently published a paper (August 2016) in Science outlining a transfer learning methodology for predicting energy consumption with satellite imagery. The researchers modified a pre-trained ImageNet classifier (vgg16) to predict nighttime light intensities that correspond with daytime satellite imagery inputs. They then extracted a rich feature space from the modified CNN, reduced dimensionality with PCA, and performed ridge regression to predict household income. The same principles could be applied to predict PM readings beyond the scope of Clarity's sensors. 

You can check out the paper at this link:
http://science.sciencemag.org/content/353/6301/790